In [1]:
import torch

In [2]:
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

In [3]:
from scipy import sparse
temp_sparse_ = sparse.load_npz( "domain1_X_y_csr.npz" )
temp_loaded = temp_sparse_.toarray()
n_samples, n_features = temp_loaded.shape
n_features -= 1 #Since the last column is actually the label
X = temp_loaded[:,:n_features]
y = temp_loaded[:,n_features]
del temp_loaded, temp_sparse_

In [4]:
mps_device = torch.device("mps")

In [5]:
import numpy as np

In [69]:
from scipy import sparse
temp_sparse_ = sparse.load_npz( "domain1_X_y_csr.npz" )
temp_loaded = temp_sparse_.toarray()
n_samples, n_features = temp_loaded.shape
n_features -= 1 #Since the last column is actually the label
X = temp_loaded[:,:n_features]
y = temp_loaded[:,n_features]
del temp_loaded, temp_sparse_

In [70]:
from torch import nn
from torch.functional import F
from torch.utils.data import DataLoader, TensorDataset

In [71]:
dataset = TensorDataset(torch.from_numpy(X).unsqueeze(2).type(torch.float32), torch.from_numpy(y).type(torch.float32))
dataloader = DataLoader(dataset)

In [64]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(6177, 6177),
            nn.ReLU(),
            nn.Linear(6177, 6177),
            nn.ReLU(),
            nn.Linear(6177, 100),
            nn.ReLU(),
            nn.Linear(100, 2)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(mps_device)
print(model)
    

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=6177, out_features=6177, bias=True)
    (1): ReLU()
    (2): Linear(in_features=6177, out_features=6177, bias=True)
    (3): ReLU()
    (4): Linear(in_features=6177, out_features=100, bias=True)
    (5): ReLU()
    (6): Linear(in_features=100, out_features=2, bias=True)
  )
)


In [65]:
# X = torch.rand(1,6177, device=mps_device)
# logits = model(X)
# pred_probab = nn.Sigmoid()(logits)
# y_pred = pred_probab.argmax(1)
# print(f"Predicted class: {y_pred}")

Predicted class: tensor([0], device='mps:0')


In [66]:
for name, param in model.named_parameters():
    nn.init.constant(param, 0)

/var/folders/pn/8q7lcn491b5gv4nk1m_p0ggm0000gn/T/ipykernel_64342/1724244856.py:2: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(param, 0)


In [85]:
torch.from_numpy(X).size()

torch.Size([5000, 6177])

In [91]:
logits = model( torch.from_numpy(X).unsqueeze(2).type(torch.float32).to(mps_device) )
loss = nn.functional.binary_cross_entropy_with_logits( logits, torch.from_numpy(y) )

ValueError: Target size (torch.Size([5000])) must be the same as input size (torch.Size([5000, 2]))

In [53]:
logits.size()

torch.Size([1, 2])